In [1]:
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [2]:
portal_client = get_portal_client()

In [3]:
df = portal_client.query_to_pandas(
    """SELECT DISTINCT test_template.site_id, test_template.test_id, test_template.template_id, test_template.created_at
    FROM (
        SELECT
            DISTINCT test.id AS test_id, test.site_id, test.created_at, task_template.template_id, task_template.asset_type
        FROM topaz.api_service_tests AS test
        INNER JOIN topaz.api_service_tasks_record AS task_record
        ON test.id = task_record.test_id
        INNER JOIN topaz.api_service_tasks AS task_template
        ON task_record.task_id = task_template.id
        AND test.created_at BETWEEN task_template.created_at AND IFNULL(task_template.deleted_at, CURRENT_TIMESTAMP())
        WHERE task_template.asset_type != ''
    ) AS test_template
    LEFT JOIN topaz.api_service_tasks AS task_template
    ON test_template.template_id = task_template.template_id
    AND (
        test_template.asset_type = task_template.asset_type
        OR task_template.asset_type = ''
    )
    AND test_template.created_at BETWEEN task_template.created_at AND IFNULL(task_template.deleted_at, CURRENT_TIMESTAMP())
    LEFT JOIN topaz.api_service_tasks_record AS task_record
    ON task_record.task_id = task_template.id
    AND task_record.test_id = test_template.test_id
    WHERE task_template.type IN (1, 3)
    AND task_record.id IS NULL
    """
)

In [4]:
for template_id, template_df in df.groupby("template_id"):
    print(template_id)
    break

10


In [5]:
template_df

,site_id,test_id,template_id,created_at
1654,22857,11957753,10,2024-12-09 06:59:27
1674,23341,12958143,10,2025-03-31 07:11:44
1675,23341,12958151,10,2025-03-31 07:13:11
1728,60930,14038566,10,2025-07-15 10:05:51
